In [1]:
# Installing dependencies 
! pip install skipthoughts

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
import pandas as pd
from nltk.util import ngrams

In [2]:
import skipthoughts
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\vinee\anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...


In [3]:
def external_features(headline, body):

  train_headline_sentences = []
  train_body_sentences = []
  train_headline_sentences.append(headline)
  train_body_sentences.append(body)

  ext = []
  i = 0
  for sent1,sent2 in zip(train_headline_sentences,train_body_sentences):
    print(i)
    i+=1
    vec = []
    #character ngrams
    for n in range(2,17):
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_2 = ngrams(sent2.lower(),n)
      vec.append(len(list(set(n_grams_1).intersection(n_grams_2))))

    #word ngrams
    for n in range(2,7):
      n_grams_1 = ngrams(sent1.lower().split(), n)
      n_grams_2 = ngrams(sent2.lower().split(),n)
      vec.append(len(list(set(n_grams_1).intersection(n_grams_2))))
    vec.append(0)
    return np.array(vec).reshape((1,21))
#     #Sentence polarity
#     flag=False
#     text1 = Text(sent1)
#     text2 = Text(sent2)
#     pol1 = 0
#     pol2 = 0
#     for word in text1.words:
#       try:
#         pol1+=word.polarity
#       except:
#         flag=True
#         vec.append(0)
#         break
#     if (flag==True):
#       ext.append(vec)
#       continue
#     flag=False
#     for word in text2.words:
#       try:
#         pol2+=word.polarity
#       except:
#         flag=True
#         vec.append(0)
#         break
#     if (flag==True):
#       ext.append(vec)
#       continue
    
    
#     pol1 = pol1/(len(sent1.split())*1.0)
#     pol2 = pol2/(len(sent2.split())*1.0)
#     print("pol1 : ", pol1)
#     print("pol2 : ", pol2)
#     print(pol1-pol2)
#     vec.append(pol1-pol2)

#     ext.append(vec)

#   return np.array(ext)

In [4]:
def statistical_features(h, b):
    
  dataset = pd.read_csv('D:\\FinalTrainSet.csv')
  headline = dataset['Headline']
  body = dataset['Body']

  v1 = TfidfVectorizer()
  statistical_h = v1.fit(headline)
  arr = statistical_h.transform([h])
  v2 = TfidfVectorizer(max_features=(10000-arr.shape[1]))
  statistical_b = v2.fit(body)
  brr = statistical_b.transform([b])
#   #b.shape
#   statistical_h = np.append(a, b).reshape((1, 2261))
#   print(statistical_h.shape)
#   print(statistical_b.shape)
#   c = statistical_b.toarray()
#   #c.shape[1]
#   d = np.zeros((1, 10000 - 2261 - c.shape[1]))
#   #d.shape
#   statistical_b = np.append(c, d).reshape((1, 10000-2261))
#   #print(statistical_b.shape)
#   print(arr.shape)
#   print(brr.shape)
  final_statistical_features = np.concatenate((arr.toarray(), brr.toarray()),axis = 1)
  print(final_statistical_features.shape)

  return final_statistical_features

In [5]:
def neural_features(h,b):
    neural_head = encoder.encode([h])
    neural_body = encoder.encode([b])
    neural_head = neural_head[0,0:2400].reshape((1,2400))
    neural_body =neural_body[0,0:2400].reshape((1,2400))
#     print(neural_head.shape)
#     print(neural_body.shape)
    final_neural_features = np.concatenate((neural_head,neural_body),axis=1)
#     print(final_neural_features.shape)
    return final_neural_features

In [6]:
from tensorflow import keras
model = keras.models.load_model('D:\\final.h5') # change the location of the model to './path/to/model/on/your/system'

In [121]:
head_demo_sent = 'ISIL Beheads American Photojournalist in Iraq'
body_demo_sent = 'James Foley, an American journalist who went missing in Syria more than a year ago, has reportedly been executed by the Islamic State, a militant group formerly known as ISIS.Video and photos purportedly of Foley emerged on Tuesday. A YouTube video -- entitled "A Message to #America (from the #IslamicState)" -- identified a man on his knees as "James Wright Foley," and showed his execution.This is a developing story. Check back here for updates.'

In [122]:
crr = statistical_features(head_demo_sent, body_demo_sent)
drr = external_features(head_demo_sent, body_demo_sent)
err = neural_features(head_demo_sent,body_demo_sent)

(1, 10000)
0
2
2


In [123]:
y_pred = model.predict(x=[err,drr,crr])

In [124]:
y_pred_ = np.argmax(y_pred,axis=1)

In [125]:
if(y_pred_[0] == 0):
    print("Headline and body pairs agree with each other")
elif(y_pred_[0] == 1):
    print("Headline and body pairs disagree with each other")
elif(y_pred_[0]==2):
    print("Headline and body pairs are discussion statements")
elif(y_pred_[0]==3):
    print("Headline and body pairs are unrelated")

Headline and body pairs agree with each other
